In [1]:
import os
os.chdir("C:/Users/Anushka/Desktop/anushka/ML/projects_vscode/IPL prediction system")

In [3]:
import pandas as pd
import os

# Load merged delivery data
df = pd.read_csv("data_processing/processed_data/merged.csv")

# Ensure match dates are sorted
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['batter', 'date'])


C:\Users\Anushka\AppData\Local\Temp\ipykernel_1300\2669094290.py:5: DtypeWarning: Columns (17,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data_processing/processed_data/merged.csv")


In [4]:
# --- Batting Trends (Last 5 Matches Average Runs) ---
batting_trends = df.groupby(['batter', 'match_id']).agg({
    'batsman_runs': 'sum',
    'date': 'first'
}).reset_index()

batting_trends['avg_runs_last_5'] = (
    batting_trends.groupby('batter')['batsman_runs']
    .transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
)

# --- Bowling Trends (Last 5 Matches Average Wickets) ---
df['wicket'] = df['is_wicket'].apply(lambda x: 1 if x == 1 else 0)

bowling_trends = df.groupby(['bowler', 'match_id']).agg({
    'wicket': 'sum',
    'date': 'first'
}).reset_index()

bowling_trends['avg_wickets_last_5'] = (
    bowling_trends.groupby('bowler')['wicket']
    .transform(lambda x: x.shift().rolling(window=5, min_periods=1).mean())
)


In [5]:
batting_trends.rename(columns={'batter': 'player'}, inplace=True)
bowling_trends.rename(columns={'bowler': 'player'}, inplace=True)

player_trends = pd.merge(batting_trends[['player', 'match_id', 'avg_runs_last_5']],
                         bowling_trends[['player', 'match_id', 'avg_wickets_last_5']],
                         on=['player', 'match_id'], how='outer')


In [6]:
os.makedirs("data_processing/processed_data", exist_ok=True)
player_trends.to_csv("data_processing/processed_data/player_trends.csv", index=False)
print("✅ player_trends.csv saved.")

✅ player_trends.csv saved.
